In [0]:
!pip install eli5
!pip install plotly_express

In [0]:
import pandas as pd
import numpy as np
pd.options.display.max_colwidth = 200

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
df = pd.read_json('train.json').drop('id', axis=1)
print(df.shape)
df.head()

(39774, 2)


,cuisine,ingredients
0,greek,"[romaine lettuce, black olives, grape tomatoes, garlic, pepper, purple onion, seasoning, garbanzo beans, feta cheese crumbles]"
1,southern_us,"[plain flour, ground pepper, salt, tomatoes, ground black pepper, thyme, eggs, green tomatoes, yellow corn meal, milk, vegetable oil]"
2,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, green chilies, grilled chicken breasts, garlic powder, yellow onion, soy sauce, butter, chicken livers]"
3,indian,"[water, vegetable oil, wheat, salt]"
4,indian,"[black pepper, shallots, cornflour, cayenne pepper, onions, garlic paste, milk, butter, salt, lemon juice, water, chili powder, passata, oil, ground cumin, boneless chicken skinless thigh, garam m..."


In [0]:
df['ingredients'] = df['ingredients'].apply(lambda x: ' '.join(x))

In [0]:
import plotly_express as px
cuisine_counts = df.groupby('cuisine').aggregate({'ingredients':np.count_nonzero}).reset_index().rename(columns = {'ingredients':'recipe_count'})
px.bar(cuisine_counts,x='cuisine',y='recipe_count',color='cuisine')

### Train/Test split

In [0]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, train_size=0.80, test_size=0.20, stratify=df['cuisine'], random_state=42)
train.shape, test.shape

((31819, 2), (7955, 2))

### Arrange data into X features matrix and y target vector

In [0]:
target = 'cuisine'
X_train=train.drop(columns=target)
y_train=train[target]
X_test=test.drop(columns=target)
y_test=test[target]
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((31819, 1), (31819,), (7955, 1), (7955,))

### Count Vectorizer along with Logistic regression model

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
vec = CountVectorizer()
clf = LogisticRegression()

pipe = make_pipeline(vec, clf)
pipe.fit(X_train.ingredients, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.



Pipeline(memory=None,
         steps=[('countvectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('logisticregression',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='warn', n_jobs=None,


In [0]:
from sklearn import metrics

def print_report(pipe):
    y_actuals = y_test
    y_preds = pipe.predict(X_test['ingredients'])
    report = metrics.classification_report(y_actuals, y_preds)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_actuals, y_preds)))

print_report(pipe)

              precision    recall  f1-score   support

   brazilian       0.72      0.59      0.65        93
     british       0.60      0.46      0.52       161
cajun_creole       0.78      0.69      0.73       309
     chinese       0.81      0.85      0.83       535
    filipino       0.71      0.56      0.62       151
      french       0.57      0.61      0.59       529
       greek       0.76      0.66      0.70       235
      indian       0.85      0.91      0.88       601
       irish       0.67      0.42      0.52       133
     italian       0.80      0.89      0.84      1568
    jamaican       0.85      0.78      0.81       105
    japanese       0.79      0.70      0.74       284
      korean       0.81      0.75      0.77       166
     mexican       0.88      0.92      0.90      1288
    moroccan       0.83      0.68      0.75       164
     russian       0.69      0.39      0.50        98
 southern_us       0.70      0.79      0.74       864
     spanish       0.69    

In [0]:
clf.classes_

array(['brazilian', 'british', 'cajun_creole', 'chinese', 'filipino',
       'french', 'greek', 'indian', 'irish', 'italian', 'jamaican',
       'japanese', 'korean', 'mexican', 'moroccan', 'russian',
       'southern_us', 'spanish', 'thai', 'vietnamese'], dtype=object)

### Feature Weights

In [0]:
for i, tag in enumerate(clf.classes_):
    coefficients = clf.coef_[i]
    weights = list(zip(vec.get_feature_names(),coefficients))
    print('Tag:',tag)
    print('Most Positive Coefficients:')
    print(sorted(weights,key=lambda x: -x[1])[:10])
    print('Most Negative Coefficients:')
    print(sorted(weights,key=lambda x: x[1])[:10])
    print("--------------------------------------")

Tag: brazilian
Most Positive Coefficients:
[('cachaca', 5.890445814166117), ('manioc', 3.170121795556176), ('tapioca', 2.9905448868968763), ('açai', 2.7192975389243457), ('sprinkles', 2.032979446449359), ('stone', 1.7866716200521304), ('piri', 1.7513929823714602), ('palm', 1.6982284528058103), ('dende', 1.6852285881453315), ('hearts', 1.4282690867488832)]
Most Negative Coefficients:
[('evaporated', -2.070566898395168), ('tortillas', -1.6705276773884858), ('curry', -1.6600785657797308), ('basil', -1.5841917003636632), ('cardamom', -1.5101676507926811), ('kidney', -1.4604257700124788), ('sesame', -1.4534732367875722), ('mushrooms', -1.4476456378316116), ('soy', -1.4083834388947913), ('seasoning', -1.3987080919947739)]
--------------------------------------
Tag: british
Most Positive Coefficients:
[('stilton', 4.8500075149848465), ('mincemeat', 3.0491621535293727), ('marmite', 2.4396943896753878), ('haddock', 2.3799185893905643), ('drippings', 2.321966250789993), ('cheddar', 2.02784109443

In [0]:
import eli5
eli5.show_weights(clf, vec=vec, top=20)